In [1]:
#IMPORTS

import matplotlib
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
import scipy.interpolate
import mne
import scipy
import statistics
from pywt import wavedec
from scipy import signal
import matplotlib.colors as mcolors
import random
import seaborn as sns


In [2]:
#DEF
#DEF UPPER CHANNEL
def upperchanel(chanlesnames):
    upperchanels=[]
    for i in chanlesnames:
        r = i.upper()
        upperchanels.append(r)
    return upperchanels

#DEF GRUP INFO DE CADA COMPONENT
def group_inf(nomgroup, data_chan, upperchanels):
    group_date = []

    for e in nomgroup:
        if e in upperchanels:
            num = upperchanels.index(e)
            #print(num)
            group_date.append(data_chan[num])  
    #print(group_date)
    return group_date

#DEF MEDIUM DELS GRUPS
#group_dat = [[1,2,3,..., 119345],[1,2,3,..., 119345],[1,2,3,..., 119345],[1,2,3,..., 119345]]
def mediumchanels(group_dat):
    segundo = len(group_dat[0])
    primero = len(group_dat)
    mean_data = []
    for i in range (0,segundo):
        suma1=[]
        for e in range(0,primero):
            suma1.append(group_dat[e][i])
        numpero = statistics.median(suma1)
        mean_data.append(numpero)
    return mean_data


def opteciogrups(chanlesnames, groupp, data_chan):
    group_date_finale = [] #Save all informatio chanels 12.
    group_date_final = [] #final copy
    upchan = upperchanel(chanlesnames)
    for i in groupp:
        group_date = group_inf(i, data_chan, upchan)
        mean_group = mediumchanels(group_date)
        mean_array = np.array(mean_group)
        group_date_finale.append(mean_array)
    group_date_final = np.array(group_date_finale)
    return group_date_final
    


In [3]:
#general info
#GROUPS
group1= ["FP2", "AFZ", "FZ", "FCZ", "CZ"]
group2= ["F7", "F5", "F3", "F1", "AF7", "AF3", "FP1"]
group3= ["FP2", "AF4", "AF8", "F2", "F4", "F6", "F8"]
group4= ["FT7", "T7", "TP7"]
group5= ["FC5", "FC3", "FC1", "C5", "C3", "C1"]
group6= ["CP5", "CP3", "CP1", "P7", "P5", "P3", "P1"]
group7= ["PO7", "PO3", "O1", "PO9"]
group8= ["PO4", "PO8", "O2", "PO10"]
group9= ["P2", "P4", "P6", "P8", "CP2", "CP4", "CP6"]
group10= ["FT8", "T8", "TP8"]
group11= ["CPZ", "PZ", "POZ", "OZ", "IZ"]
group12= ["FC2", "FC4", "FC6", "C2", "C4", "C6"]

#X
groups = [group1, group2, group3, group4, group5, group6, group7, group8, group9, group10, group11, group12]

In [4]:
#LOAD
x2=mne.io.read_raw_eeglab('Files/Preprocessed/sub-010003_EC.set', preload=True, verbose=True)
#GET DATA
datax2 = x2._data
#GET CHANELS
chanles_names2 = x2.ch_names
#REDUCE CHANELS TO 12
groups_date_finalle2 = opteciogrups(chanles_names2, groups, datax2)
#TRANSFORM TO MNE
titlegrup = ["ch1","ch2","ch3","ch4","ch5","ch6","ch7","ch8","ch9","ch10","ch11","ch12"]
ct_ty = ["eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg"]
sfreq = 250
info = mne.create_info(ch_names=titlegrup, sfreq=sfreq, ch_types=ct_ty)
raw2 = mne.io.RawArray(groups_date_finalle2, info)

Reading Files/Preprocessed/sub-010003_EC.fdt
Reading 0 ... 119515  =      0.000 ...   478.060 secs...
Creating RawArray with float64 data, n_channels=12, n_times=119516
    Range : 0 ... 119515 =      0.000 ...   478.060 secs
Ready.


In [5]:
#ALL

import math
data = groups_date_finalle2[0]

Fs = 250.0
Ts = 1.0/Fs
t = np.arange(len(data)) / Fs

n = len(data) # length of the signal
k = np.arange(n)
T = n/Fs
frq = k/T # two sides frequency range
frq = frq[range(int(n/2))]

Y = np.fft.fft(data)/n
Y = Y[range(int(n/2))]


fig, ax = plt.subplots(2, 1)
ax[0].plot(t,data)
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Amplitude')
ax[1].plot(frq,abs(Y),'r')
ax[1].set_xlabel('Freq (Hz)')

plt.draw()
plt.show()

In [6]:
len(data)

119516

In [7]:
t

array([0.00000e+00, 4.00000e-03, 8.00000e-03, ..., 4.78052e+02,
       4.78056e+02, 4.78060e+02])

478.064

In [19]:
#epoch fuction (5s split with overlap)(n epochs x)
# for each epoch power sepctral density
colors = mcolors.BASE_COLORS 
sf=250


tinici = 0
tfinal = 1250
printsome = 0
r=0

save_epoc_data = []
save_epoc_temps = []

def find_nearest(array,value):
       idx = (np.abs(array-value)).argmin()
       return int(idx)

for i in range(0,int(T)):

    epoc_temps = t[tinici:tfinal-1]
    epoc_data = groups_date_finalle2[1,tinici:tfinal-1]

    print(len(epoc_temps))
    print(len(epoc_data))


    if printsome<=-2:
        plt.plot(epoc_temps,epoc_data) 
        plt.show()

        x = epoc_data
        coeffs = wavedec(x, 'db4', level=6)

        #
        Fs = 250.0
        Ts = 1.0/Fs
        tn = np.arange(len(x)) / Fs
        n = len(x) # length of the signal
        k = np.arange(n)
        Tn = n/Fs
        frq = k/Tn # two sides frequency range
        frq = frq[range(int(n/2))]
        Y = np.fft.fft(x)/n
        Y = Y[range(int(n/2))]
        fig, ax = plt.subplots(2, 1)
        ax[0].plot(tn,x)
        ax[0].set_xlabel('Time')
        ax[0].set_ylabel('Amplitude')
        ax[1].plot(frq,abs(Y),'r')
        ax[1].set_xlabel('Freq (Hz)')

        #frq
        tf = find_nearest(frq, 4)
        ti = find_nearest(frq, 45)
        powerv = x[ti:tf]
        #https://numpy.org/doc/stable/reference/generated/numpy.trapz.html 
        #tr = numpy.trapz( ,powerv) #big band
        #find the "small" band
        #Divided each one with the big band

        plt.draw()
        plt.show()

        '''# Define window length (4 seconds)
        #signal.welch <- search measure
        win = .31 * sf
        freqs, psd = signal.welch(x, sf, nperseg=win)

        # Plot the power spectrum (les time 1s + overlap)
        sns.set(font_scale=1, style='white')
        #plt.figure(figsize=(8, 4))
        plt.plot(freqs, psd, color=random.choice(list(colors.values())), lw=2)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power spectral density (V^2 / Hz)')
        plt.ylim([0, psd.max() * 8])
        plt.title("Welch's periodogram")
        plt.xlim([0, freqs.max()])
        sns.despine()'''

        f, Pxx_spec = signal.welch(x, 250, 'flattop', 1024, scaling='density')
        plt.figure()
        plt.semilogy(f, np.sqrt(Pxx_spec))
        plt.xlabel('frequency [Hz]')
        plt.ylabel('Linear spectrum [V RMS]')
        plt.title('Power spectrum (scipy.signal.welch)')
        plt.show()

        
        #cD5 - b.Power - power spectral density - (after - cut it in frec. and find the average, see it under the spectrum).


    printsome=printsome+1


    tfinal=tfinal+1250
    tinici= tinici+1250


1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
1249
766
766
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [10]:
#https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html

In [18]:
frq

array([  0.        ,   0.20016013,   0.40032026,   0.60048038,
         0.80064051,   1.00080064,   1.20096077,   1.4011209 ,
         1.60128102,   1.80144115,   2.00160128,   2.20176141,
         2.40192154,   2.60208167,   2.80224179,   3.00240192,
         3.20256205,   3.40272218,   3.60288231,   3.80304243,
         4.00320256,   4.20336269,   4.40352282,   4.60368295,
         4.80384307,   5.0040032 ,   5.20416333,   5.40432346,
         5.60448359,   5.80464371,   6.00480384,   6.20496397,
         6.4051241 ,   6.60528423,   6.80544436,   7.00560448,
         7.20576461,   7.40592474,   7.60608487,   7.806245  ,
         8.00640512,   8.20656525,   8.40672538,   8.60688551,
         8.80704564,   9.00720576,   9.20736589,   9.40752602,
         9.60768615,   9.80784628,  10.00800641,  10.20816653,
        10.40832666,  10.60848679,  10.80864692,  11.00880705,
        11.20896717,  11.4091273 ,  11.60928743,  11.80944756,
        12.00960769,  12.20976781,  12.40992794,  12.61

In [11]:
#standar erros mean
#filter 30hz? paper check
# - preproz bandpass filtered within 1-45 Hz (8th order, Butterworth filter)
# - (vi) band-pass filtering between 0.01-0.1 Hz (FSL),

# Gamma

# http://www.sci.utah.edu/publications/anderson10/Anderson_python_processing2010.pdf
#-



# cut it to diferent frec. bands theta (4-8Hz), alpha(8-13Hz), beta(13-30Hz), gamma(30-45Hz).
#-

#aver. ex/4-8 for power values -> 1 feature x epoch x sub. x brain region (chanels 12).
#all up / all bands  (to se the band power %)